<a href="https://colab.research.google.com/github/cheolhakja/fine-dust-prediction/blob/main/gnn/gnn_preprocess_202506.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
from geopy.distance import geodesic
import glob
import numpy as np
from sklearn.preprocessing import StandardScaler



drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).




### 1. 공기 측정소에 가장 가까운 기상 측정소를 매핑한다



In [ ]:
"""
aws_ station 메타데이터에서, 유효한 관측소만 추출한다 (운영 중지된 관측소는 제거함)
"""
aws_station = pd.read_csv('/content/drive/MyDrive/graduation_project/aws_station_metadata.csv', encoding='euc-kr')

aws_station = aws_station[aws_station['종료일'].isna()].copy()



In [ ]:
air_station = pd.read_csv('/content/drive/MyDrive/graduation_project/도시_도로변_입체_측정소위치.csv')

air_station = air_station[air_station['sido']=='서울'].copy()

air_station_modified = air_station.copy()

# 가장 가까운 aws_station의 지점명과 지점을 저장할 리스트
nearest_station_list = []
nearest_id_list = []

# 각 air_station 행에 대해 가장 가까운 aws_station 찾기
for idx, row in air_station_modified.iterrows():
    lat, lon = row['dmX'], row['dmY']

    # 거리 계산 함수
    def wgs84_distance(x):
        return geodesic((lat, lon), (x['위도'], x['경도'])).km

    # aws_station의 모든 행에 대해 거리 계산
    aws_station['distance'] = aws_station.apply(wgs84_distance, axis=1)

    # 가장 가까운 aws_station의 지점명과 지점 찾기
    nearest_row = aws_station.nsmallest(1, 'distance')
    nearest_station = nearest_row['지점명'].iloc[0]
    nearest_id = nearest_row['지점'].iloc[0]

    nearest_station_list.append(nearest_station)
    nearest_id_list.append(nearest_id)

# 새로운 칼럼으로 추가
air_station_modified['지점명'] = nearest_station_list
air_station_modified['지점'] = nearest_id_list

air_station = air_station_modified.copy()

air_station.iloc[:10]

,sido,stationName,addr,dmX,dmY,지점명,지점
0,서울,중구,서울 중구 덕수궁길 15 시청서소문별관 3동,37.564639,126.975961,서울,108
1,서울,한강대로,서울 용산구 한강대로 405 (서울역 앞),37.549389,126.971519,중구,419
2,서울,종로구,"서울 종로구 종로35가길 19 종로5,6가 동 주민센터",37.572025,127.005028,중구,419
3,서울,청계천로,서울 중구 청계천로 184 (청계천4가사거리 남강빌딩 앞),37.568650,126.998083,중구,419
4,서울,종로,서울 종로구 종로 169 (종묘주차장 앞),37.570633,126.996783,중구,419
5,서울,용산구,서울특별시 용산구 이태원로 224-19 (한남동) 한남로 복합문화센터,37.532057,127.002371,중구,419
6,서울,광진구,서울특별시 광진구 광나루로 571 구의 아리수정수센터,37.544639,127.095706,광진,413
7,서울,성동구,서울 성동구 뚝섬로3길 18 성수1가1동주민센터,37.542036,127.049685,성동,421
8,서울,강변북로,서울 성동구 강변북로 257 한강사업본부 옆,37.539283,127.040943,성동,421
9,서울,중랑구,서울 중랑구 용마산로 369 건강가정지원센터,37.584953,127.094283,중랑,409


### 2. 공기/기상 측정소 정보와, 미세먼지 측정값을 매핑하기

In [ ]:
"""미세먼지 측정값 데이터프레임에서 필요없는 칼럼 제거
"""

df_air_june = pd.read_csv('/content/drive/MyDrive/graduation_project/air_202506.csv')

air_col = [
    'so2Value', 'coValue', 'o3Value', 'no2Value', 'pm10Value', 'pm25Value','stationName','datetime'
]

df_air_june = df_air_june[air_col].copy()



# 조인 수행 (stationName 기준, left join)
df_air_june_merged = df_air_june.merge(
    air_station[['stationName', 'dmX', 'dmY', '지점명', '지점']],
    on='stationName',
    how='left'
)

# 결과 확인 (선택 사항)
print("조인된 데이터프레임 상위 5행:")
print(df_air_june_merged.head())

# df_air_june 업데이트
df_air_june = df_air_june_merged.copy()

# 조인 후 데이터 확인
print(f"조인 후 행 수: {len(df_air_june)}")
print("조인된 칼럼:", df_air_june.columns.tolist())


조인된 데이터프레임 상위 5행:
  so2Value coValue o3Value no2Value pm10Value pm25Value stationName  \
0   0.0019    0.18  0.0097   0.0115         7         7          중구   
1   0.0022    0.19  0.0121   0.0108         7         4          중구   
2    0.003    0.19  0.0162   0.0114        11         8          중구   
3   0.0026     0.2  0.0206    0.012        11         9          중구   
4   0.0031    0.21  0.0265    0.013         9         8          중구   

              datetime        dmX         dmY 지점명     지점  
0  2025-06-30 23:00:00  37.564639  126.975961  서울  108.0  
1  2025-06-30 22:00:00  37.564639  126.975961  서울  108.0  
2  2025-06-30 21:00:00  37.564639  126.975961  서울  108.0  
3  2025-06-30 20:00:00  37.564639  126.975961  서울  108.0  
4  2025-06-30 19:00:00  37.564639  126.975961  서울  108.0  
조인 후 행 수: 114230
조인된 칼럼: ['so2Value', 'coValue', 'o3Value', 'no2Value', 'pm10Value', 'pm25Value', 'stationName', 'datetime', 'dmX', 'dmY', '지점명', '지점']


### 1분 단위 aws데이터를 '1시간 간격의 평균' 으로 바꾸기

In [ ]:
tmp2 = pd.read_csv('/content/drive/MyDrive/graduation_project/aws_1min_202506.csv')

# 0) 일시를 datetime 으로
tmp2['일시'] = pd.to_datetime(tmp2['일시'])

# 1) 풍향·풍속 → 벡터 성분 u, v
wd_rad = np.deg2rad(tmp2['풍향(deg)'])
tmp2['u'] = tmp2['풍속(m/s)'] * np.cos(wd_rad)
tmp2['v'] = tmp2['풍속(m/s)'] * np.sin(wd_rad)

# 2) 1 시간 단위로 그룹핑(지점별)
tmp3 = (
    tmp2
    .set_index('일시')                       # 시간 인덱스
    .groupby('지점')
    .resample('1H')
    .agg({
        '풍속(m/s)'      : 'mean',          # 1h 평균 풍속
        'u'              : 'mean',          # 벡터 평균용
        'v'              : 'mean',
        '1분 강수량(mm)' : 'sum',           # 1h 누적 강수
        '기온(°C)'       : 'mean'           # 1h 평균 기온
    })
    .reset_index()
)

# 3) 벡터 평균 풍향 계산 후 컬럼 정리
tmp3['풍향(deg)_1h'] = (
    np.degrees(np.arctan2(tmp3['v'], tmp3['u'])) % 360
)

# 필요 없는 중간 성분 제거(선택)
tmp3.drop(columns=['u', 'v'], inplace=True)

# 컬럼 이름 보기 좋게 변경(선택)
tmp3.rename(columns={
    '풍속(m/s)'      : '풍속(m/s)_1h',
    '1분 강수량(mm)' : '강수량(mm)_1h',
    '기온(°C)'       : '기온(°C)_1h'
}, inplace=True)

# 결과 확인
print(tmp3.iloc[:10])


/tmp/ipython-input-15-1877927570.py:16: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample('1H')


    지점                  일시  풍속(m/s)_1h  강수량(mm)_1h  기온(°C)_1h  풍향(deg)_1h
0  116 2025-06-01 00:00:00    0.952542         0.0  17.406780  355.380764
1  116 2025-06-01 01:00:00    1.211667         0.0  17.245000    5.637634
2  116 2025-06-01 02:00:00    0.840000         0.0  17.200000    2.507692
3  116 2025-06-01 03:00:00    0.638333         0.0  17.125000  348.304057
4  116 2025-06-01 04:00:00    0.513333         0.0  17.990000   16.550469
5  116 2025-06-01 05:00:00    0.218333         0.0  18.688333  275.724537
6  116 2025-06-01 06:00:00    0.366667         0.0  18.723333  261.541618
7  116 2025-06-01 07:00:00    0.128333         0.0  19.448333  313.386205
8  116 2025-06-01 08:00:00    0.113333         0.0  20.208333  358.657579
9  116 2025-06-01 09:00:00    0.048333         0.0  20.753333  340.690948


In [ ]:
"""아래와 같이 하면 **지점코드(칼럼명: 지점 )와 시간(예: 일시)**으로
df_air_june과 tmp를 조인(merge) 할 수 있습니다."""

# 1. 시간 컬럼명, 타입 맞추기
df_air_june = df_air_june.rename(columns={'datetime': '일시'})
df_air_june['일시'] = pd.to_datetime(df_air_june['일시'])
tmp3['일시'] = pd.to_datetime(tmp3['일시'])
df_air_june = df_air_june[df_air_june['지점'].notna()] #결측치 날리기


# 2. 지점 컬럼 타입도 맞추기 (둘 다 int 혹은 str로 통일!)
# 여기선 int로 맞추는 게 일반적
df_air_june['지점'] = df_air_june['지점'].astype(int)
tmp3['지점'] = tmp3['지점'].astype(int)

# 3. LEFT JOIN 수행
df_joined = pd.merge(
    df_air_june,
    tmp3,
    how='left',
    on=['지점', '일시'],
    suffixes=('', '_tmp')
)

print(df_joined.head())


  so2Value coValue o3Value no2Value pm10Value pm25Value stationName  \
0   0.0019    0.18  0.0097   0.0115         7         7          중구   
1   0.0022    0.19  0.0121   0.0108         7         4          중구   
2    0.003    0.19  0.0162   0.0114        11         8          중구   
3   0.0026     0.2  0.0206    0.012        11         9          중구   
4   0.0031    0.21  0.0265    0.013         9         8          중구   

                   일시        dmX         dmY 지점명   지점  풍속(m/s)_1h  강수량(mm)_1h  \
0 2025-06-30 23:00:00  37.564639  126.975961  서울  108         NaN         NaN   
1 2025-06-30 22:00:00  37.564639  126.975961  서울  108         NaN         NaN   
2 2025-06-30 21:00:00  37.564639  126.975961  서울  108         NaN         NaN   
3 2025-06-30 20:00:00  37.564639  126.975961  서울  108         NaN         NaN   
4 2025-06-30 19:00:00  37.564639  126.975961  서울  108         NaN         NaN   

   기온(°C)_1h  풍향(deg)_1h  
0        NaN         NaN  
1        NaN         NaN  
2    

In [ ]:
"""결측 제거"""
# 1. 결측치 제거 전 row 수
print("제거 전 row 수:", len(df_joined))

# 2. 모든 컬럼 중 하나라도 NaN 있으면 제거
df_clean = df_joined.dropna()

# 3. 결측치 제거 후 row 수
print("제거 후 row 수:", len(df_clean))


제거 전 row 수: 27600
제거 후 row 수: 26904


### 데이터 정규화: Z-score

In [ ]:
feature_cols = [
    'so2Value', 'coValue', 'o3Value', 'no2Value', 'pm10Value', 'pm25Value',
    '풍속(m/s)_1h', '강수량(mm)_1h', '기온(°C)_1h', '풍향(deg)_1h'
]

# 1. feature 컬럼 전부 숫자로 강제 변환
for col in feature_cols:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# 2. 정규화 전에 결측치 제거(모든 컬럼에 NaN 있으면 제거)
df_clean = df_clean.dropna(subset=feature_cols)


# df_clean이 결측치 없는 최종 데이터라고 가정
scaler = StandardScaler()

# feature 컬럼만 정규화, 나머지 컬럼(일시, stationName 등)은 그대로 두고, 정규화된 데이터프레임을 만듦
df_clean_scaled = df_clean.copy()
df_clean_scaled[feature_cols] = scaler.fit_transform(df_clean[feature_cols])

# 결과 확인
print(df_clean_scaled.head())


     so2Value   coValue   o3Value  no2Value  pm10Value  pm25Value stationName  \
690 -0.596133 -0.412573 -1.263414 -0.442108  -0.551999  -1.074997        한강대로   
691 -0.596133 -0.061284 -1.198088 -0.370889  -0.152175  -0.789683        한강대로   
692 -0.596133  0.114360 -1.202171 -0.238625  -0.285450  -0.599474        한강대로   
693 -0.596133  0.026538 -1.079684 -0.401412  -0.152175  -0.314160        한강대로   
694 -0.758849 -0.149106 -0.777550 -0.228451  -0.618637  -0.504369        한강대로   

                     일시        dmX         dmY 지점명   지점  풍속(m/s)_1h  \
690 2025-06-30 23:00:00  37.549389  126.971519  중구  419   -1.063243   
691 2025-06-30 22:00:00  37.549389  126.971519  중구  419   -1.159879   
692 2025-06-30 21:00:00  37.549389  126.971519  중구  419   -0.855654   
693 2025-06-30 20:00:00  37.549389  126.971519  중구  419   -1.091876   
694 2025-06-30 19:00:00  37.549389  126.971519  중구  419   -1.088297   

     강수량(mm)_1h  기온(°C)_1h  풍향(deg)_1h  
690    -0.14593   0.498111   -0.254734  
691 

/tmp/ipython-input-19-3936071730.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')


In [ ]:
"""
save result
"""
df_clean_scaled.to_csv('/content/drive/MyDrive/gnn/gnn_2d_202506.csv', index=False)